In [1]:
import numpy as np
import cv2 as cv

In [2]:
!pip install -e /home/lars/devel/tpk4170-robotics/

Obtaining file:///home/lars/devel/tpk4170-robotics
  Found existing installation: tpk4170 1.0
    Uninstalling tpk4170-1.0:
      Successfully uninstalled tpk4170-1.0
  Running setup.py develop for tpk4170
You are using pip version 10.0.1, however version 18.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [3]:
np.set_printoptions(suppress=True)

In [4]:
from tpk4170.visualization import Viewer
from tpk4170.models import Ball, Grid, Axes, Plane, Line, Triangle
from tpk4170.utils.transformations import quaternion_from_matrix, unit_vector

In [5]:
from tpk4170.pose_estimation import solveP3P

# The Perspective-3-Point Problem

## Camera intrinsic parameters

In [6]:
camera_matrix = np.array([[750.0, 0.0, 1280/2],
                          [0.0, 750.0, 1024/2],
                          [0.0, 0.0, 1.0]])
print(camera_matrix)

[[750.   0. 640.]
 [  0. 750. 512.]
 [  0.   0.   1.]]


In [7]:
dist_coeffs = np.zeros((1, 5))
print(dist_coeffs)

[[0. 0. 0. 0. 0.]]


## Camera extrinsic parameters

In [8]:
tvec = np.array([0.5, 0.5, 2.0])
print(tvec)

[0.5 0.5 2. ]


In [9]:
R = np.array([[0.0, -1.0, 0.0, 0.0],
              [-1.0, 0.0, 0.0, 0.0],
              [0.0, 0.0, -1.0, 0.0],
              [0.0, 0.0, 0.0, 1.0]])
print(R[:3,:3])

[[ 0. -1.  0.]
 [-1.  0.  0.]
 [ 0.  0. -1.]]


In [10]:
rvec, _ = cv.Rodrigues(R[:3, :3])
print(rvec)

[[ 2.22144147]
 [-2.22144147]
 [ 0.        ]]


## Object points

In [11]:
obj_points = np.array([[0.5, 0.5, 0.0],
                       [0.5, 1.5, 0.0],
                       [1.5, 1.5, 0.0],
                       [1.5, 0.5, 0.3]]) / 2
print(obj_points)

[[0.25 0.25 0.  ]
 [0.25 0.75 0.  ]
 [0.75 0.75 0.  ]
 [0.75 0.25 0.15]]


## Image points (Pixel coordinates)

In [12]:
img_points, _ = cv.projectPoints(
    obj_points, rvec, tvec, camera_matrix, dist_coeffs)
print(img_points.reshape(-1,2))

[[733.75       605.75      ]
 [546.25       605.75      ]
 [546.25       418.25      ]
 [741.35135135 410.64864865]]


## Homogeneous image points  (Pixel coordinates)

In [13]:
hom_img_points = np.hstack((img_points.reshape(-1, 2), np.ones((4, 1))))
print(hom_img_points)

[[733.75       605.75         1.        ]
 [546.25       605.75         1.        ]
 [546.25       418.25         1.        ]
 [741.35135135 410.64864865   1.        ]]


## Normalized Homogenous Image Points (Metric coordinates)

In [14]:
norm_img_points = (np.linalg.inv(camera_matrix) @ hom_img_points.T).T
print(norm_img_points)

[[ 0.125       0.125       1.        ]
 [-0.125       0.125       1.        ]
 [-0.125      -0.125       1.        ]
 [ 0.13513514 -0.13513514  1.        ]]


## Visualization

In [15]:
viewer = Viewer()
viewer.add(Grid())
viewer.add(Axes(0.5))

Renderer(camera=PerspectiveCamera(aspect=1.5, children=(DirectionalLight(color='white', intensity=0.66, positi…

### Add camera frame

In [17]:
cam_axes = Axes(0.5)
cam_axes.position = tvec.tolist()
cam_axes.quaternion = quaternion_from_matrix(R).tolist()
viewer.add(cam_axes)

### Add object points in world frame

In [19]:
for obj_point in obj_points:
    ball = Ball()
    ball.position = obj_point.tolist()
    viewer.add(ball)

### Add projection lines

In [20]:
for obj_point in obj_points:
    line = Line(tvec, obj_point)
    viewer.add(line)

### Add triangle of object_points

In [21]:
obj_triangle = Triangle(*obj_points[:3])
viewer.add(obj_triangle)

### Add triangle of normalized homogenous image points

In [22]:
img_triangle = Triangle(*norm_img_points[:3])
cam_axes.add(img_triangle)

## Solutions

Find $n$ solutions using `cv.solveP3P`:

In [23]:
n, rvecs, tvecs = cv.solveP3P(
    obj_points[:3, :], img_points[:3, :], camera_matrix, dist_coeffs, cv.SOLVEPNP_AP3P)
print('Found {} solution(s):\n'.format(n))
for rvec, tvec in zip(rvecs, tvecs):
    T = np.eye(4)
    R, _ = cv.Rodrigues(rvec)
    t = tvec
    T[:3,:3] = R
    T[:3,3] = t.ravel()
    print(T,'\n')

Found 4 solution(s):

[[-0.03030303 -0.96969697  0.24242424  0.5       ]
 [-0.96969697 -0.03030303 -0.24242424  0.5       ]
 [ 0.24242424 -0.24242424 -0.93939394  2.        ]
 [ 0.          0.          0.          1.        ]] 

[[-0.01146663 -0.94295857 -0.33271257  0.46402284]
 [-0.99571761  0.04129242 -0.08271257  0.46402284]
 [ 0.09173304  0.33033934 -0.93939394  1.69781421]
 [ 0.          0.          0.          1.        ]] 

[[ 0.  -1.  -0.   0.5]
 [-1.   0.  -0.   0.5]
 [ 0.   0.  -1.   2. ]
 [ 0.   0.   0.   1. ]] 

[[ 0.04129242 -0.99571761  0.08271257  0.49040236]
 [-0.94295857 -0.01146663  0.33271257  0.49040236]
 [-0.33033934 -0.09173304 -0.93939394  2.11988658]
 [ 0.          0.          0.          1.        ]] 



Find $n$ solutions using solveP3P from this package:

In [29]:
# n, rvecs, tvecs = solveP3P(
#     obj_points, img_points, camera_matrix, dist_coeffs)
# print('Found {} solution(s):\n'.format(n))
# for rvec, tvec in zip(rvecs, tvecs):
#     T = np.eye(4)
#     R, _ = cv.Rodrigues(rvec)
#     t = tvec
#     T[:3,:3] = R
#     T[:3,3] = t.ravel()
#     print(T,'\n')

### Visualize Solutions

In [27]:
viewer = Viewer()
viewer.add(Grid())
viewer.add(Axes(0.5))
for obj_point in obj_points:
    ball = Ball()
    ball.position = obj_point.ravel().tolist()
    viewer.add(ball)

Renderer(camera=PerspectiveCamera(aspect=1.5, children=(DirectionalLight(color='white', intensity=0.66, positi…

In [28]:
def visualize_solution(viewer, obj_points, img_points, rvec, tvec):
    tvec = tvec.ravel()
    R = np.eye(4)
    R[:3, :3] = cv.Rodrigues(rvec)[0]

    cam_axes = Axes(0.5)
    cam_axes.position = tvec.tolist()
    cam_axes.quaternion = quaternion_from_matrix(R).tolist()
    viewer.add(cam_axes)

#     img_points, _ = cv.projectPoints(
#         obj_points, rvec, tvec, camera_matrix, dist_coeffs)
#     hom_img_points = np.hstack((img_points.reshape(-1, 2), np.ones((4, 1))))
#     norm_img_points = (np.linalg.inv(camera_matrix) @ hom_img_points.T).T
    
#     print(norm_img_points)

#     img_triangle = Triangle(*norm_img_points[:3])
#     cam_axes.add(img_triangle)
    
    
#     for obj_point in obj_points:
#         line = Line(tvec, obj_point)
#         viewer.add(line)

### Add all solutions to viewer

In [29]:
for rvec, tvec in zip(rvecs, tvecs):
    visualize_solution(viewer, obj_points, img_points, rvec, tvec)